In [ ]:
!pip install torch torchvision

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
!pip install torchinfo

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

In [ ]:
print(f'Torch version: {torch.__version__}')

Torch version: 2.3.0+cu121


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# mean and std according to https://pytorch.org/vision/main/models/generated/torchvision.models.efficientnet_b0.html#torchvision.models.EfficientNet_B0_Weights
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

In [ ]:
train_transform = transforms.Compose([
                                transforms.Resize(256),
                                transforms.RandomResizedCrop(224),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize(mean, std)])

test_transform = transforms.Compose([
                                transforms.Resize(256),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize(mean, std)])

In [ ]:
trainset = torchvision.datasets.Food101(root='./drive/My Drive/efficientnet_b0/datasets',
                                        split ='train',
                                        download=True,
                                        transform=train_transform)

100%|██████████| 4996278331/4996278331 [08:18<00:00, 10022293.82it/s]


Extracting ./drive/My Drive/efficientnet_b0/datasets/food-101.tar.gz to ./drive/My Drive/efficientnet_b0/datasets


In [ ]:
testset = torchvision.datasets.Food101(root='./drive/My Drive/efficientnet_b0/datasets',
                                       split='test',
                                       download=True,
                                       transform=train_transform)

In [ ]:
batch_size = 250
num_workers = 2

In [ ]:
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=num_workers)

In [ ]:
testloader = torch.utils.data.DataLoader(testset,
                                         batch_size=batch_size,
                                         shuffle=False,
                                         num_workers=num_workers)

In [ ]:
print(trainset.classes)

['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito', 'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake', 'ceviche', 'cheese_plate', 'cheesecake', 'chicken_curry', 'chicken_quesadilla', 'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder', 'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes', 'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict', 'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras', 'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari', 'fried_rice', 'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad', 'grilled_cheese_sandwich', 'grilled_salmon', 'guacamole', 'gyoza', 'hamburger', 'hot_and_sour_soup', 'hot_dog', 'huevos_rancheros', 'hummus', 'ice_cream', 'lasagna', 'lobster_bisque', 'lobster_roll_sandwich', 'macaroni_and_cheese', 'macarons', 'miso_sou

In [ ]:
model = models.efficientnet_b0(weights='DEFAULT')

In [ ]:
from torchinfo import summary
summary(model)

Layer (type:depth-idx)                                  Param #
EfficientNet                                            --
├─Sequential: 1-1                                       --
│    └─Conv2dNormActivation: 2-1                        --
│    │    └─Conv2d: 3-1                                 864
│    │    └─BatchNorm2d: 3-2                            64
│    │    └─SiLU: 3-3                                   --
│    └─Sequential: 2-2                                  --
│    │    └─MBConv: 3-4                                 1,448
│    └─Sequential: 2-3                                  --
│    │    └─MBConv: 3-5                                 6,004
│    │    └─MBConv: 3-6                                 10,710
│    └─Sequential: 2-4                                  --
│    │    └─MBConv: 3-7                                 15,350
│    │    └─MBConv: 3-8                                 31,290
│    └─Sequential: 2-5                                  --
│    │    └─MBConv: 3-9         

In [ ]:
num_features = model.classifier[1].in_features
print(f'The number of features on the pretrained model: {num_features}')

The number of features on the pretrained model: 1280


In [ ]:
num_food_classes = len(trainset.classes)
model.fc = nn.Linear(num_features, num_food_classes)

In [ ]:
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(),
                       lr=learning_rate)

In [ ]:
exp_lr_scheduler = lr_scheduler.StepLR(optimizer,
                                       step_size=10,
                                       gamma=0.9)

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
def compute_average_validation_loss_and_accuracy(model,
                                    testloader,
                                    criterion,
                                    device):
  model.eval()
  val_loss = 0.0
  correct= 0
  total = 0
  with torch.no_grad():
    for inputs, labels in testloader:
      inputs, labels = inputs.to(device), labels.to(device)

      outputs = model(inputs)
      loss = criterion(outputs, labels)
      val_loss += loss.item() * inputs.size(0)

      _, predicted = torch.max(outputs, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
  average_val_loss = val_loss / len(testloader.dataset)
  accuracy = correct/total
  return (average_val_loss, accuracy)

In [ ]:
def show_loss_and_accuracy_details(epoch_number,
                                   total_number_epochs,
                                   model,
                                   testloader,
                                   criterion,
                                   device):
  average_val_loss, accuracy = compute_average_validation_loss_and_accuracy(model, testloader, criterion, device)
  print(f'Epoch [{epoch+1}/{total_number_epochs}], Validation Loss: {average_val_loss:.4f}, Accuracy: {accuracy:.4f}')

In [ ]:
def save_checkpoint_model(epoch,
                          model_state_dict,
                          optimizer_state_dict,
                          loss,
                          checkpoint_path):
  checkpoint_path = f'{checkpoint_path}model{epoch}.pt'
  torch.save({
      'epoch': epoch,
      'model_state_dict': model_state_dict,
      'optimizer_state_dict': optimizer_state_dict,
      'loss': loss
  }, checkpoint_path)

In [ ]:
checkpoint_path = './drive/My Drive/efficientnet_b0/checkpoints/'

In [ ]:
num_training_epochs = 50

In [ ]:
model.to(device)
for epoch in range(num_training_epochs):
  print(f'Training - epoch {epoch + 1}.')

  model.train()
  for inputs, labels in trainloader:
    inputs, labels = inputs.to(device), labels.to(device)

    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

  if(epoch % 3 == 0):
    save_checkpoint_model(epoch, model.state_dict(), optimizer.state_dict(), loss, checkpoint_path)
    show_loss_and_accuracy_details(epoch, num_training_epochs, model, testloader, criterion, device)

Training - epoch 1.
Epoch [1/50], Validation Loss: 1.2605, Accuracy: 0.6716
Training - epoch 2.
Training - epoch 3.
Training - epoch 4.
Epoch [4/50], Validation Loss: 0.9264, Accuracy: 0.7527
Training - epoch 5.
Training - epoch 6.
Training - epoch 7.
Epoch [7/50], Validation Loss: 0.8644, Accuracy: 0.7702
Training - epoch 8.
Training - epoch 9.
Training - epoch 10.
Epoch [10/50], Validation Loss: 0.8410, Accuracy: 0.7752
Training - epoch 11.
Training - epoch 12.
Training - epoch 13.
Epoch [13/50], Validation Loss: 0.8409, Accuracy: 0.7800
Training - epoch 14.
Training - epoch 15.
Training - epoch 16.
Epoch [16/50], Validation Loss: 0.8309, Accuracy: 0.7880
Training - epoch 17.
Training - epoch 18.
Training - epoch 19.
Epoch [19/50], Validation Loss: 0.8437, Accuracy: 0.7881
Training - epoch 20.
Training - epoch 21.
Training - epoch 22.
Epoch [22/50], Validation Loss: 0.8529, Accuracy: 0.7872
Training - epoch 23.
Training - epoch 24.
Training - epoch 25.
Epoch [25/50], Validation Loss:

In [ ]:
model.eval()
with torch.no_grad():
  _ , accuracy = compute_average_validation_loss_and_accuracy(model, testloader, criterion, device)
  print(f'Accuracy of the model on the test images: {accuracy*100:.4f}')

Accuracy of the model on the test images: 78.6931


In [ ]:
torch.save(model, './drive/My Drive/efficientnet_b0/efficientnet_food101model.pt')

In [ ]:
!pip install onnx
!pip install onnxscript

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 625.5/625.5 kB 17.0 MB/s eta 0:00:00


In [ ]:
sample_input = torch.randn(1, 3, 224, 224).cuda()

onnx_file_path = "./drive/My Drive/efficientnet_b0/efficientnet_food101model.onnx"

torch.onnx.export(model, sample_input, onnx_file_path)

In [ ]:
!nvidia-smi

Wed Jun 12 18:41:53 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0              55W / 400W |  28817MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--